In [1]:
import pymongo
import pandas as pd
#from pandas import Series
import numpy as np
import soundfile as sf
#import matplotlib.pyplot as plt
#from collections import Counter
from bson.objectid import ObjectId
import json
import pprint
from pydub import AudioSegment
import urllib
import os
import requests
import ffmpy
import subprocess

/home/rrs432/miniconda2/envs/bandhub/lib/python2.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
client = pymongo.MongoClient('localhost',27017)
#client = pymongo.MongoClient('localhost',32768)
db = client.bandhub

In [3]:
songCol = db.get_collection('songsStream')
vidCol = db.get_collection('mergedVideos')
trackCol = db.get_collection('tracksStream')
postCol = db.get_collection('posts')
[songCol.count(), vidCol.count(), trackCol.count(), postCol.count()]

[425706, 198169, 350000, 494867]

In [17]:
TwoTrackMixes = []
#zeroTrackMixes = 0
pubTracks = postCol.find({'songAccess':{'$exists': True}}, {'participantsInfo':1 ,'songAccess':1, '_id':1})
for docs in pubTracks:
    if(docs['songAccess'] == 1):
        if(len(docs['participantsInfo']['publishedTracks']) == 2):
            TwoTrackMixes.append(docs['_id'])
#pprint.pprint(len(TwoTrackMixes))

remix_test1 = TwoTrackMixes[0]
pprint.pprint(remix_test1)

ObjectId('546f05de4e5d9e60267e62da')


In [4]:
ThreeTrackMixes = []
masterSongIDs = []
#zeroTrackMixes = 0
pubTracks = postCol.find({'songAccess':{'$exists': True}}, {'participantsInfo':1 ,'songAccess':1, 'objectId':1})
for docs in pubTracks:
    if(docs['songAccess'] == 1):
        if(len(docs['participantsInfo']['publishedTracks']) == 3):
            #ThreeTrackMixes.append(docs['objectId'])
            ThreeTrackMixes.append(docs['_id'])
            masterSongIDs.append(docs['objectId'])
#pprint.pprint(len(TwoTrackMixes))

remix_test = ThreeTrackMixes[5]
songID = masterSongIDs[5]
pprint.pprint(songID)

ObjectId('5346e55daadff46b2c000126')


In [5]:
pub_t = []
rmt1 = postCol.find({'_id': remix_test})
count = 0
for docs in rmt1:
    if docs['songAccess'] == 1:
        #pprint.pprint(docs)
        #pprint.pprint(docs['participantsInfo']['publishedTracks'])
        count = len(docs['participantsInfo']['publishedTracks'])
        #pprint.pprint(docs['participantsInfo']['publishedTracks'])
        for i in range(count):
            pub_t.append(docs['participantsInfo']['publishedTracks'][i]['_id'])
        #pprint.pprint("Not public")
pprint.pprint(pub_t)

[ObjectId('5346e6c5aadff46b2c00012b'),
 ObjectId('535db15e9d193e5f03000066'),
 ObjectId('53a0e215b722525949000005')]


In [6]:
songLookUp = songCol.find({'masterSongId' : songID})
pub_t_vol = []
#pprint.pprint(songLookUp.count())
for docs in songLookUp:
    #pprint.pprint(docs)
    for i in range(count):
        #pprint.pprint(docs['settings'][str(pub_t[i])]['volume'])
        #pprint.pprint(str(pub_t[i]))
        #pprint.pprint(docs['settings'][str(pub_t[i])]['volume'])
        pub_t_vol.append(docs['settings'][str(pub_t[i])]['volume'])
    #print('found')
pprint.pprint(pub_t_vol)

[1, 1, 0.7]


In [7]:
ogg_paths = []
track_filenames = []
start_times = []
for i in range(count):
    gettrack = trackCol.find({'_id' : pub_t[i]})
    for docs in gettrack:
        #trackVideo = docs['videoFileUrl']
        #pprint.pprint(docs['audioChannels'][0]['fileUrl'])
        ogg_paths.append(docs['audioChannels'][0]['fileUrl'])
        start_times.append(docs['startTimeValue'])
        #pprint.pprint(docs)
#pprint.pprint(ogg_path)
for file in ogg_paths:
    filePath = file.split('/')
    track_filenames.append(filePath[- 1])
    #print fileName
pprint.pprint(track_filenames)
pprint.pprint(start_times)

[u'578feca1-696d-4d49-bb75-15dc63bb5822-0-0_surubi.ogg',
 u'136dd451-e101-49a9-a7db-edef3f1ef098-0-0.ogg',
 u'3258166c-a1ee-4f6a-9876-2d9325287a82-0.ogg']
[0, 0, 0]


In [8]:
path = '/scratch/rrs432/tracks'
#os.mkdir(path)
os.chdir(path)

for i in range(count):
    r = requests.get(ogg_paths[i])
    filename = ogg_paths[i].rpartition('/')[2]
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
print('Download successful')

Download successful


In [9]:
pointer = vidCol.find({'songId': songID})
for songdocs in pointer:
    premix = songdocs['oggMergedVideoUrl']
pprint.pprint(premix)

u'http://bandhubmediastream.blob.core.windows.net/files/f-baede503-caf6-4029-a777-84b2a5421878.ogg'


In [10]:
path = '/scratch/rrs432/tracks'
#os.mkdir(path)
os.chdir(path)

pointer = vidCol.find({'songId': songID})
for songdocs in pointer:
    premix = songdocs['oggMergedVideoUrl']

r = requests.get(premix)
filename = premix.rpartition('/')[2]
with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)
print('BandHub Mix Download successful')

BandHub Mix Download successful


In [11]:
ls

136dd451-e101-49a9-a7db-edef3f1ef098-0-0.ogg
3258166c-a1ee-4f6a-9876-2d9325287a82-0.ogg
578feca1-696d-4d49-bb75-15dc63bb5822-0-0_surubi.ogg
f-baede503-caf6-4029-a777-84b2a5421878.ogg
gconcr/
heyoh/


In [12]:
oggTracks = []
trackLens = []
samRates = []
for i in range(count):
    oggTrack, samRate = sf.read('/scratch/rrs432/tracks/'+track_filenames[i])
    oggTracks.append(oggTrack)
    samRates.append(samRate)
    trackLens.append(len(oggTracks[i])) 
pprint.pprint(trackLens)
#pprint.pprint(len(oggTracks[2]))
#pprint.pprint(np.shape(samRates))

[11120205, 11210560, 10363456]


In [13]:
if (min(samRates) == max(samRates)):
    mixsr = samRates[0] 
else:
    pprint.pprint('Warning: Sample rates unequal')
pprint.pprint(mixsr)

44100


In [14]:
masterLen = max(trackLens)
pprint.pprint(masterLen)

11210560


In [15]:
for i in range(count):
    pprint.pprint(len(oggTracks[i]))
    if(trackLens[i] != masterLen):
        #pprint.pprint(start_times[i])
        if(start_times[i] == 0):
            diff = np.zeros(masterLen - trackLens[i])
            #pprint.pprint(len(oggTracks[i]))
            oggTracks[i] = np.concatenate((oggTracks[i], diff), axis=0)
            #pprint.pprint(masterLen - trackLens[i])
            pprint.pprint(len(oggTracks[i]))
            #pprint.pprint(len(diff))
            if(len(oggTracks[i]) == masterLen):
                pprint.pprint('Successful')
            else:
                pprint.pprint('Try again')
        else:
            diff = np.zeros(start_times[i]-1)
            #pprint.pprint(len(diff))
            oggTracks[i] = np.concatenate((diff,oggTracks[i]), axis=0)
            diff = np.zeros(masterLen - len(oggTracks[i]))
            oggTracks[i] = np.concatenate((oggTracks[i], diff), axis=0)
            pprint.pprint(len(oggTracks[i]))
    else:
        pprint.pprint('Equal')

11120205
11210560
'Successful'
11210560
'Equal'
10363456
11210560
'Successful'


In [16]:
# path = '/scratch/rrs432/tracks'
# #os.mkdir(path)
# os.chdir(path)
mixOgg = 0
for i in range(count):
    #pprint.pprint('lol')
    #pprint.pprint(pub_t_vol[i])
    #pprint.pprint(oggTracks[i])
    mixOgg += (pub_t_vol[i]*np.array(oggTracks[i]))
sf.write('mixed.wav', mixOgg / np.max(np.abs(mixOgg)), mixsr)

In [17]:
path = '/scratch/rrs432/tracks'
os.chdir(path)

In [84]:
!which ffmpeg

which: no ffmpeg in (/home/rrs432/miniconda2/envs/bandhub/bin:/share/apps/mongodb/3.4.10/bin:/home/rrs432/miniconda2/envs/bandhub/bin:/home/rrs432/miniconda2/bin:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/ibutils/bin:/share/apps/local/bin:/share/apps/centos/7/bin:/opt/slurm/bin:/opt/dell/srvadmin/bin:/home/rrs432/.local/bin:/home/rrs432/bin)


In [81]:
subprocess.call(['ffmpeg', '-i', 'mixed.wav', '-codec:a', 'libvorbis', '-b:a', '232k', 'remixed.ogg'])

OSError: [Errno 2] No such file or directory

In [28]:
!which ffmpeg

which: no ffmpeg in (/home/rrs432/miniconda2/envs/bandhub/bin:/share/apps/mongodb/3.4.10/bin:/home/rrs432/miniconda2/envs/bandhub/bin:/home/rrs432/miniconda2/bin:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/ibutils/bin:/share/apps/local/bin:/share/apps/centos/7/bin:/opt/slurm/bin:/opt/dell/srvadmin/bin:/home/rrs432/.local/bin:/home/rrs432/bin)


In [26]:
!ffmpeg -i 'mixed.wav' -codec:a libvorbis -b:a 232k remixed.ogg

/usr/bin/sh: ffmpeg: command not found


In [18]:
!ls

136dd451-e101-49a9-a7db-edef3f1ef098-0-0.ogg	     gconcr	premix.ogg
3258166c-a1ee-4f6a-9876-2d9325287a82-0.ogg	     heyoh	remixed.ogg
578feca1-696d-4d49-bb75-15dc63bb5822-0-0_surubi.ogg  mixed.wav


In [20]:
p, sr = sf.read('premixa.ogg')
r, sr = sf.read('remixed.ogg')
#pprint.pprint(np.shape(p)[1])
if np.shape(p)[1] == 2:
    p = p.sum(axis=1) / 2
pprint.pprint(np.shape(p))
pprint.pprint(np.shape(r))

(11212096,)
(11210560,)


In [21]:
pLen = len(p)
rLen = len(r)
if pLen < rLen:
    diff = np.zeros(rLen - pLen)
    p = np.concatenate((p,diff),axis = 0)
else:
    diff = np.zeros(pLen - rLen)
    r = np.concatenate((r,diff),axis = 0)
pprint.pprint(len(diff))
pprint.pprint(len(r))
pprint.pprint(len(p))

1536
11212096
11212096


In [22]:
np.allclose(p,r)

False

In [28]:
path = '/scratch/rrs432/tracks'
#os.mkdir(path)
os.chdir(path)
mixogg = 0.7*oggtrack1 + 0.7*oggtrack2 + 0.7*oggtrack3
sf.write('mixed.wav', mixogg / np.max(np.abs(mixogg)), mixsr)
#sf.write('mixed.ogg', mixogg, mixsr)

In [49]:
ls

8f127b81-5d41-1b61-c028-e769fbfe85c9.ogg
d4f684e0-1989-4abd-913a-5e78edebfe32-0-0.ogg
f-1827b0e1-9972-4c9f-bfe3-88db569ce857.ogg
mixed.ogg


In [50]:
audiodiff.audio_equal('mixed.ogg', 'f-1827b0e1-9972-4c9f-bfe3-88db569ce857.ogg')

OSError: [Errno 2] No such file or directory

In [13]:
np.shape(ogg_final)

(8419264, 2)

In [48]:
pointer = trackCol.find({'_id' : ObjectId('520bfa263004aac9f04d42dd')})
for docs in pointer:
    pprint.pprint(docs['audioChannels'][0]['fileUrl'])

u'http://bandhubwebmedia1.blob.core.windows.net/files/520bf9573004aac9f04d42db-520bfa263004aac9f04d42dd.ogg'


In [49]:
pointer = trackCol.find({'_id' : ObjectId('549707e68cdf4b53785ab214')})
for docs in pointer:
    pprint.pprint(docs['audioChannels'][0]['fileUrl'])

u'http://bandhubwebmedia8.blob.core.windows.net/files/14c676d9-891b-4179-ab10-8e6fd86327eb-0.ogg'


In [50]:
pointer = trackCol.find({'_id' : ObjectId('54970d1c8cdf4b53785ab85e')})
for docs in pointer:
    pprint.pprint(docs['audioChannels'][0]['fileUrl'])

u'http://bandhubwebmedia2.blob.core.windows.net/files/ca8eee7b-9c91-4853-aa1f-b84130cae7c7-0.ogg'


In [51]:
pointer = vidCol.find()[2:3]
for docs in pointer:
    pprint.pprint(docs)

{u'_id': ObjectId('52ea92cce4b0a865287a1352'),
 u'mp4MergedNewsWebVideoUrl': u'https://bandhubmediastream.blob.core.windows.net/files/f-ff1751e2-893e-4e27-9acc-49d14a3157b3.mp4',
 u'mp4MergedVideoUrl': u'https://bandhubmediastream.blob.core.windows.net/files/f-52ea87fb080a26053f000002.mp4',
 u'oggMergedVideoUrl': u'https://bandhubmediastream.blob.core.windows.net/files/f-52ea87fb080a26053f000002.ogg',
 u'songId': ObjectId('52ea87fb080a26053f000002'),
 u'trackIds': [ObjectId('52ea88fca3c2473745000001'),
               ObjectId('52ea8d6ca3c2473745000004')],
 u'youtubeId': u'aXdF95VBGFE'}


In [24]:
pointer = songCol.find()[10:11]
for docs in pointer:
    if docs['access'] == 1:
        pprint.pprint(docs)
    else:
        pprint.pprint("Not public")

{u'_id': ObjectId('520bf7af3004aac9f04d42cc'),
 u'access': 1,
 u'communityOrder': 300,
 u'dateLastModified': datetime.datetime(2013, 10, 12, 4, 25, 53, 39000),
 u'description': u'',
 u'fbImage': u'http://bandhubmediastream.blob.core.windows.net/files/ct-520bf7af3004aac9f04d42cc-5.png',
 u'fbImageTracks': [u'520bf8b63004aac9f04d42ce',
                    u'53a7827cc6542a8b7c000006',
                    u'53d76e02f03678007700028a',
                    u'53dd7ecca4135c842a004b54'],
 u'hasRecordedTracks': True,
 u'inCommunity': True,
 u'isFeatured': False,
 u'masterSongId': ObjectId('522a1cc26310675e3200004e'),
 u'migratedToAzure': True,
 u'mp4MergedMobileVideoUrl': u'http://bandhubmediastream.blob.core.windows.net/files/m-f-5b5f21ea-2d44-46b3-99e3-25cdede0a3fd.mp4',
 u'mp4MergedVideoUrl': u'http://bandhubmediastream.blob.core.windows.net/files/f-ec9545b5-3acd-43d6-8f7b-3754fc2cc7db.mp4',
 u'musicbrainzMetadataId': ObjectId('544a0452e4b0031ed66d0aba'),
 u'musicbrainzMetadataLookup': True,


In [25]:
pointer = vidCol.find()[2:3]
for docs in pointer:
    pprint.pprint(docs)

{u'_id': ObjectId('52ea92cce4b0a865287a1352'),
 u'mp4MergedNewsWebVideoUrl': u'https://bandhubmediastream.blob.core.windows.net/files/f-ff1751e2-893e-4e27-9acc-49d14a3157b3.mp4',
 u'mp4MergedVideoUrl': u'https://bandhubmediastream.blob.core.windows.net/files/f-52ea87fb080a26053f000002.mp4',
 u'oggMergedVideoUrl': u'https://bandhubmediastream.blob.core.windows.net/files/f-52ea87fb080a26053f000002.ogg',
 u'songId': ObjectId('52ea87fb080a26053f000002'),
 u'trackIds': [ObjectId('52ea88fca3c2473745000001'),
               ObjectId('52ea8d6ca3c2473745000004')],
 u'youtubeId': u'aXdF95VBGFE'}


In [ ]:
masterLen = max(oggtracks)
diff = 0
if((np.shape(oggtrack1)[0]) < (np.shape(oggtrack2)[0])):
    mixogg = np.zeros(np.shape(oggtrack2)[0])
    diff = np.zeros(np.shape(oggtrack2)[0] - np.shape(oggtrack1)[0])
    oggtrack1 = np.concatenate((oggtrack1, diff), axis=0)
else:
    mixogg = np.zeros(np.shape(oggtrack1)[0])
    diff = np.zeros(np.shape(oggtrack1)[0] - np.shape(oggtrack2)[0])
    oggtrack2 = np.concatenate((oggtrack2, diff), axis=0)
if((np.shape(oggtrack1)[0]) < (np.shape(oggtrack3)[0])):
    mixogg = np.zeros(np.shape(oggtrack3)[0])
    diff = np.zeros(np.shape(oggtrack3)[0] - np.shape(oggtrack1)[0])
    oggtrack1 = np.concatenate((oggtrack1, diff), axis=0)
else:
    mixogg = np.zeros(np.shape(oggtrack1)[0])
    diff = np.zeros(np.shape(oggtrack1)[0] - np.shape(oggtrack3)[0])
    oggtrack3 = np.concatenate((oggtrack3, diff), axis=0) 
#pprint.pprint(np.shape(diff))
pprint.pprint(np.shape(oggtrack1))
pprint.pprint(np.shape(oggtrack2))
pprint.pprint(np.shape(oggtrack3))

In [5]:
data = pd.read_hdf('BandhubDatasetCopy.h5')

In [12]:
data.iloc[2]

trackId                                               54357096e18d72165e42ba85
songId                                                542fcd68e4251e5e66ec1cb2
masterOwner                                                             Brandx
trackOwner                                                              Brandx
artist                                                             Iron Maiden
title                                                      back in the village
views                                                                      800
instrument                                                                 NaN
contentTags                                                               null
audioURL                     http://bandhubwebmedia4.blob.core.windows.net/...
cleanProcessedAudioURL       http://bandhubwebmedia5.blob.core.windows.net/...
processedAudioURL                                                          NaN
startTime                                           

In [ ]:
path = '/home/rrs432/'
#os.mkdir(path)
os.chdir(path)